<a href="https://colab.research.google.com/github/Normal-case/MechineLearning/blob/master/jeju_creditCard_dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 로드

In [ ]:
#그래프의 전체적인 설정을 하는 객체
import matplotlib as mpl
#그래프를 그리는 객체
import matplotlib.pyplot as plt
#그래프의 글자를 선명하게 출력하도록 설정 |
%config InlineBackend.figure_format = 'retina'
#나눔 글자체 설치 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
#나눔고딕 글자체의 전체 경로 설정
fontpath = 'd'
#글자체의 경로와 글자 크기 설정
font = fm.FontProperties(fname=fontpath, size=9)

#폰트 이름 설정
plt.rc('font', family='NanumBarunGothic') 
#폰트 업데이트
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


INFO:matplotlib.font_manager:generated new fontManager


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
path = './drive/My Drive/인공지능 큐레이터/Machine Learning/data/'
dfRest = pd.read_csv(path + '201901-202003.csv')
df04 = pd.read_csv(path + '202004.csv')

In [ ]:
df = pd.concat([dfRest, df04])
del dfRest, df04

In [ ]:
df_submission = pd.read_csv(path + 'submission.csv')
df_submission.drop(['id'], axis = 1, inplace = True)

# 전처리

## 결측치 처리

In [ ]:
df[df['CARD_SIDO_NM'].isna() | df['HOM_CCG_NM'].isna()]

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
420,201901,강원,강릉시,과실 및 채소 소매업,세종,NaN,30s,1,2,3,58000,3
421,201901,강원,강릉시,과실 및 채소 소매업,세종,NaN,40s,1,3,3,263000,4
1189,201901,강원,강릉시,기타 대형 종합 소매업,세종,NaN,20s,1,1,3,176540,3
1190,201901,강원,강릉시,기타 대형 종합 소매업,세종,NaN,20s,2,1,4,234880,6
1191,201901,강원,강릉시,기타 대형 종합 소매업,세종,NaN,30s,1,2,4,177640,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1349960,202004,충북,충주시,한식 음식점업,세종,NaN,30s,2,2,3,80000,4
1349961,202004,충북,충주시,한식 음식점업,세종,NaN,40s,1,3,16,465400,19
1349962,202004,충북,충주시,한식 음식점업,세종,NaN,50s,1,4,33,1492450,50
1349963,202004,충북,충주시,한식 음식점업,세종,NaN,50s,2,4,3,30000,3


In [ ]:
df.loc[df['CARD_SIDO_NM'] == '세종', 'CARD_CCG_NM'] = '세종시'
df.loc[df['HOM_SIDO_NM'] == '세종', 'HOM_CCG_NM'] = '세종시'
df.isna().sum()

REG_YYMM        0
CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
dtype: int64

## 이상치 처리(뛰어넘어도됨)

In [ ]:
# AMT의 IQR을 이용해서 이상치 제거
q1 = np.percentile(df['AMT'], 25)
q3 = np.percentile(df['AMT'], 75)
IQR = q3 - q1
df_outlier = df[df['AMT'] > q3 + (1.5 * IQR)] # 낮은 쪽의 이상치는 없기 때문에 높은 쪽의 이상치만 추출
group = df_outlier.groupby(by = ['CARD_SIDO_NM', 'STD_CLSS_NM']) # 시별 업종별 이상치를 찾기 위해 그룹화
group = group.count()['AMT']
group[group == 1].index # 이상치 중 개수가 하나인 것만 확인

MultiIndex([('강원',         '자동차 임대업'),
            ('광주', '그외 기타 스포츠시설 운영업'),
            ('대전',            '여행사업'),
            ('대전',         '자동차 임대업'),
            ('세종',             '여관업'),
            ('울산',             '면세점'),
            ('울산',            '여행사업'),
            ('인천',    '기타 수상오락 서비스업'),
            ('인천',         '자동차 임대업'),
            ('전북',       '내항 여객 운송업'),
            ('전북',          '버스 운송업'),
            ('충남', '그외 기타 스포츠시설 운영업')],
           names=['CARD_SIDO_NM', 'STD_CLSS_NM'])

In [ ]:
'''
삭제 할 칼럼
- 4월 -
강원 - 자동차 임대업 - 30s - 15407881 - 0
대전 - 자동차 임대업 - 50s - 12551863 - 3
울산 - 면세점 - 40s - 7895761 - 5
울산 - 여행사업 - 50s - 4486264 - 6
전북 - 내항 여객 운송업 - 50s - 1405951 - 9
전북 - 버스 운송업 - 60s - 4681779 - 10
- 3월 - 똑같음
강원 - 자동차 임대업 - 30s - 15407881 - 0
대전 - 자동차 임대업 - 50s - 12551863 - 3
울산 - 면세점 - 40s - 7895761 - 5
울산 - 여행사업 - 50s - 4486264 - 6
전북 - 내항 여객 운송업 - 50s - 1405951 - 9
전북 - 버스 운송업 - 60s - 4681779 - 10
'''

'\n삭제 할 칼럼\n- 4월 -\n강원 - 자동차 임대업 - 30s - 15407881 - 0\n대전 - 자동차 임대업 - 50s - 12551863 - 3\n울산 - 면세점 - 40s - 7895761 - 5\n울산 - 여행사업 - 50s - 4486264 - 6\n전북 - 내항 여객 운송업 - 50s - 1405951 - 9\n전북 - 버스 운송업 - 60s - 4681779 - 10\n- 3월 - 똑같음\n강원 - 자동차 임대업 - 30s - 15407881 - 0\n대전 - 자동차 임대업 - 50s - 12551863 - 3\n울산 - 면세점 - 40s - 7895761 - 5\n울산 - 여행사업 - 50s - 4486264 - 6\n전북 - 내항 여객 운송업 - 50s - 1405951 - 9\n전북 - 버스 운송업 - 60s - 4681779 - 10\n'

In [ ]:
dummy = [15407881, 12551863, 7895761, 4486264, 1405951, 4681779]
df = df.drop(dummy) # 이상치 삭제

## 구매력 칼럼

In [ ]:
# # 구매력 POWER 구하는 코드
# # 코드 실행하면 너무 오래걸려서 밑 코드 실행으로 변수입력

# colist_cardsido = df.CARD_SIDO_NM.unique().tolist()
# colist_clss = df.STD_CLSS_NM.unique().tolist()

# dic_power = {}

# for i in colist_cardsido:
#   for j in colist_clss:
#     temp1 = df[(df["CARD_SIDO_NM"]=="{}".format(i)) & (df["STD_CLSS_NM"]=="{}".format(j))].groupby("AGE").sum()[["CNT", "AMT"]].reset_index()
#     temp1["AGEPOWER"] = temp1["AMT"]/temp1["CNT"]
#     temp1["REGIONTOTALPOWER"] = temp1["CNT"]/temp1["CNT"].sum()*temp1["AGEPOWER"]
#     # globals()["{}_{}_power".format(i, j)]=temp1["REGIONTOTALPOWER"].sum()
#     dic_power['{}_{}'.format(i, j)] = temp1["REGIONTOTALPOWER"].sum()

In [ ]:
dic_power = {'강원_건강보조식품 소매업': 112121.20537811179,
 '강원_골프장 운영업': 112546.79590565234,
 '강원_과실 및 채소 소매업': 38558.158795363444,
 '강원_관광 민예품 및 선물용품 소매업': 20761.327287135682,
 '강원_그외 기타 분류안된 오락관련 서비스업': 17700.0,
 '강원_그외 기타 스포츠시설 운영업': 58204.36781609196,
 '강원_그외 기타 종합 소매업': 33698.29079162859,
 '강원_기타 대형 종합 소매업': 38333.96512903954,
 '강원_기타 수상오락 서비스업': 19524.5084837059,
 '강원_기타 외국식 음식점업': 25950.502862568228,
 '강원_기타 주점업': 102928.04510580373,
 '강원_기타음식료품위주종합소매업': 29769.114032370224,
 '강원_내항 여객 운송업': 22545.001852995676,
 '강원_마사지업': 107737.11752010987,
 '강원_면세점': 54758.86363636363,
 '강원_버스 운송업': 9273.609378897481,
 '강원_비알콜 음료점업': 8349.514983173556,
 '강원_빵 및 과자류 소매업': 11904.025857584107,
 '강원_서양식 음식점업': 16098.145532914285,
 '강원_수산물 소매업': 53911.200519099184,
 '강원_슈퍼마켓': 19615.608365448654,
 '강원_스포츠 및 레크레이션 용품 임대업': 96542.59171116294,
 '강원_여관업': 51561.26172756135,
 '강원_여행사업': 26091.51948181082,
 '강원_욕탕업': 13313.385118254826,
 '강원_육류 소매업': 42878.68130950097,
 '강원_일반유흥 주점업': 151658.62801521295,
 '강원_일식 음식점업': 62496.07066957818,
 '강원_자동차 임대업': 158443.38481012656,
 '강원_전시 및 행사 대행업': 35952.47226146122,
 '강원_정기 항공 운송업': 0.0,
 '강원_중식 음식점업': 23970.266607277445,
 '강원_차량용 가스 충전업': 43333.204505752954,
 '강원_차량용 주유소 운영업': 56806.84868211517,
 '강원_체인화 편의점': 7606.469618129547,
 '강원_택시 운송업': 12824.44976076555,
 '강원_피자 햄버거 샌드위치 및 유사 음식점업': 11054.88371693451,
 '강원_한식 음식점업': 31902.68384225201,
 '강원_호텔업': 61995.59156379024,
 '강원_화장품 및 방향제 소매업': 45072.714837380125,
 '강원_휴양콘도 운영업': 54392.11787535525,
 '경기_건강보조식품 소매업': 93396.88932552334,
 '경기_골프장 운영업': 141677.3238207895,
 '경기_과실 및 채소 소매업': 21445.946794521846,
 '경기_관광 민예품 및 선물용품 소매업': 24378.673557801027,
 '경기_그외 기타 분류안된 오락관련 서비스업': 24833.333333333336,
 '경기_그외 기타 스포츠시설 운영업': 18048.376420897464,
 '경기_그외 기타 종합 소매업': 45285.84552476809,
 '경기_기타 대형 종합 소매업': 32881.1305474721,
 '경기_기타 수상오락 서비스업': 38406.69224399668,
 '경기_기타 외국식 음식점업': 21557.15323958199,
 '경기_기타 주점업': 98655.70465401151,
 '경기_기타음식료품위주종합소매업': 28764.82277862391,
 '경기_내항 여객 운송업': 42815.13513513514,
 '경기_마사지업': 106947.41000112589,
 '경기_면세점': 0.0,
 '경기_버스 운송업': 10980.867984955417,
 '경기_비알콜 음료점업': 6639.64020515179,
 '경기_빵 및 과자류 소매업': 9973.677854447915,
 '경기_서양식 음식점업': 15016.644444287413,
 '경기_수산물 소매업': 35966.809406990775,
 '경기_슈퍼마켓': 17698.115001970753,
 '경기_스포츠 및 레크레이션 용품 임대업': 87516.86902367929,
 '경기_여관업': 42443.77883566641,
 '경기_여행사업': 20936.996575342462,
 '경기_욕탕업': 13077.356698086383,
 '경기_육류 소매업': 29464.388757725963,
 '경기_일반유흥 주점업': 98595.36030649918,
 '경기_일식 음식점업': 41604.52508936284,
 '경기_자동차 임대업': 476549.46767631103,
 '경기_전시 및 행사 대행업': 18711.482010088308,
 '경기_정기 항공 운송업': 0.0,
 '경기_중식 음식점업': 23162.009707373792,
 '경기_차량용 가스 충전업': 31711.52435514653,
 '경기_차량용 주유소 운영업': 51913.49656091505,
 '경기_체인화 편의점': 7032.935066459681,
 '경기_택시 운송업': 8470.924004794082,
 '경기_피자 햄버거 샌드위치 및 유사 음식점업': 10123.997021597255,
 '경기_한식 음식점업': 29075.31441747613,
 '경기_호텔업': 121720.22885542705,
 '경기_화장품 및 방향제 소매업': 34590.928751396095,
 '경기_휴양콘도 운영업': 49165.96075706568,
 '경남_건강보조식품 소매업': 103132.43315130912,
 '경남_골프장 운영업': 125054.54920095105,
 '경남_과실 및 채소 소매업': 32852.85510571801,
 '경남_관광 민예품 및 선물용품 소매업': 32965.242452830185,
 '경남_그외 기타 분류안된 오락관련 서비스업': 34228.94736842105,
 '경남_그외 기타 스포츠시설 운영업': 17360.52379125096,
 '경남_그외 기타 종합 소매업': 46013.26201646814,
 '경남_기타 대형 종합 소매업': 31165.1706545252,
 '경남_기타 수상오락 서비스업': 26605.68538011696,
 '경남_기타 외국식 음식점업': 31546.525497269682,
 '경남_기타 주점업': 89031.39222096765,
 '경남_기타음식료품위주종합소매업': 28416.624535821,
 '경남_내항 여객 운송업': 25687.57919103314,
 '경남_마사지업': 115512.60118617098,
 '경남_면세점': 0.0,
 '경남_버스 운송업': 7859.207485740244,
 '경남_비알콜 음료점업': 7416.862047267649,
 '경남_빵 및 과자류 소매업': 12065.410995726597,
 '경남_서양식 음식점업': 17490.28937301492,
 '경남_수산물 소매업': 59039.47290392489,
 '경남_슈퍼마켓': 17151.215600394204,
 '경남_스포츠 및 레크레이션 용품 임대업': 75524.16169396156,
 '경남_여관업': 41085.01602620624,
 '경남_여행사업': 43549.46401225115,
 '경남_욕탕업': 19146.819639358382,
 '경남_육류 소매업': 32428.17211101362,
 '경남_일반유흥 주점업': 130444.5168209528,
 '경남_일식 음식점업': 55530.81643329036,
 '경남_자동차 임대업': 189068.4463768116,
 '경남_전시 및 행사 대행업': 19576.236133249244,
 '경남_정기 항공 운송업': 0.0,
 '경남_중식 음식점업': 22603.484385902884,
 '경남_차량용 가스 충전업': 32716.974211373086,
 '경남_차량용 주유소 운영업': 57065.75729157473,
 '경남_체인화 편의점': 7390.925799580452,
 '경남_택시 운송업': 6873.7269910133255,
 '경남_피자 햄버거 샌드위치 및 유사 음식점업': 10628.63944237882,
 '경남_한식 음식점업': 31784.558693890274,
 '경남_호텔업': 104241.49065635644,
 '경남_화장품 및 방향제 소매업': 39157.62622619081,
 '경남_휴양콘도 운영업': 52877.940716569094,
 '경북_건강보조식품 소매업': 124670.97944679859,
 '경북_골프장 운영업': 107828.42169519483,
 '경북_과실 및 채소 소매업': 34407.15722260829,
 '경북_관광 민예품 및 선물용품 소매업': 17348.78402268179,
 '경북_그외 기타 분류안된 오락관련 서비스업': 33750.0,
 '경북_그외 기타 스포츠시설 운영업': 15412.782393237138,
 '경북_그외 기타 종합 소매업': 36282.41693736955,
 '경북_기타 대형 종합 소매업': 32985.46493499395,
 '경북_기타 수상오락 서비스업': 26264.93702770781,
 '경북_기타 외국식 음식점업': 30331.771973407183,
 '경북_기타 주점업': 91162.02362829792,
 '경북_기타음식료품위주종합소매업': 26476.470055637095,
 '경북_내항 여객 운송업': 46366.639209225694,
 '경북_마사지업': 98371.35571380128,
 '경북_면세점': 0.0,
 '경북_버스 운송업': 10808.857538035962,
 '경북_비알콜 음료점업': 7925.023273611067,
 '경북_빵 및 과자류 소매업': 11863.827175193703,
 '경북_서양식 음식점업': 12713.02427191407,
 '경북_수산물 소매업': 56694.99292575977,
 '경북_슈퍼마켓': 17037.89806557517,
 '경북_스포츠 및 레크레이션 용품 임대업': 93227.92626057185,
 '경북_여관업': 40453.53154163163,
 '경북_여행사업': 189266.66666666666,
 '경북_욕탕업': 17775.500717908482,
 '경북_육류 소매업': 35974.39900697496,
 '경북_일반유흥 주점업': 139297.33206570472,
 '경북_일식 음식점업': 44771.44441076582,
 '경북_자동차 임대업': 162847.2908496732,
 '경북_전시 및 행사 대행업': 16507.54607689985,
 '경북_정기 항공 운송업': 0.0,
 '경북_중식 음식점업': 22556.70031457195,
 '경북_차량용 가스 충전업': 34534.08650728801,
 '경북_차량용 주유소 운영업': 62520.722914257574,
 '경북_체인화 편의점': 7374.501811562549,
 '경북_택시 운송업': 6738.9473684210525,
 '경북_피자 햄버거 샌드위치 및 유사 음식점업': 10592.830857032077,
 '경북_한식 음식점업': 29075.574353878314,
 '경북_호텔업': 43289.11938088299,
 '경북_화장품 및 방향제 소매업': 38255.7494584646,
 '경북_휴양콘도 운영업': 65458.00863035345,
 '광주_건강보조식품 소매업': 128185.08672987325,
 '광주_골프장 운영업': 77359.30690719202,
 '광주_과실 및 채소 소매업': 31495.42172269229,
 '광주_관광 민예품 및 선물용품 소매업': 27543.274313240465,
 '광주_그외 기타 분류안된 오락관련 서비스업': 0.0,
 '광주_그외 기타 스포츠시설 운영업': 19153.697259008313,
 '광주_그외 기타 종합 소매업': 43209.18645469857,
 '광주_기타 대형 종합 소매업': 36690.142597676,
 '광주_기타 수상오락 서비스업': 23128.4046692607,
 '광주_기타 외국식 음식점업': 27608.764620448153,
 '광주_기타 주점업': 115602.55425566874,
 '광주_기타음식료품위주종합소매업': 27099.758127951412,
 '광주_내항 여객 운송업': 23107.062526227444,
 '광주_마사지업': 112358.12636229751,
 '광주_면세점': 0.0,
 '광주_버스 운송업': 10403.4942680776,
 '광주_비알콜 음료점업': 6788.2645435478635,
 '광주_빵 및 과자류 소매업': 11248.131544513124,
 '광주_서양식 음식점업': 12595.247056790313,
 '광주_수산물 소매업': 50927.48224685825,
 '광주_슈퍼마켓': 15392.309238345408,
 '광주_스포츠 및 레크레이션 용품 임대업': 101583.57270210049,
 '광주_여관업': 39601.22872832846,
 '광주_여행사업': 812832.0,
 '광주_욕탕업': 14974.975361199955,
 '광주_육류 소매업': 31820.254035753715,
 '광주_일반유흥 주점업': 105820.17392260201,
 '광주_일식 음식점업': 45886.27490371706,
 '광주_자동차 임대업': 172172.16179775278,
 '광주_전시 및 행사 대행업': 38154.34444952033,
 '광주_정기 항공 운송업': 0.0,
 '광주_중식 음식점업': 23389.715548377408,
 '광주_차량용 가스 충전업': 29889.36198357483,
 '광주_차량용 주유소 운영업': 54546.2093284983,
 '광주_체인화 편의점': 6805.679940459385,
 '광주_택시 운송업': 9570.0,
 '광주_피자 햄버거 샌드위치 및 유사 음식점업': 10272.004725098072,
 '광주_한식 음식점업': 34043.24585121719,
 '광주_호텔업': 66582.75844594595,
 '광주_화장품 및 방향제 소매업': 35812.14876585522,
 '광주_휴양콘도 운영업': 0.0,
 '대구_건강보조식품 소매업': 104245.04365062076,
 '대구_골프장 운영업': 91929.48222630174,
 '대구_과실 및 채소 소매업': 19215.415057247417,
 '대구_관광 민예품 및 선물용품 소매업': 46888.74379872431,
 '대구_그외 기타 분류안된 오락관련 서비스업': 0.0,
 '대구_그외 기타 스포츠시설 운영업': 30581.29314922995,
 '대구_그외 기타 종합 소매업': 43960.13841471741,
 '대구_기타 대형 종합 소매업': 32644.637392768964,
 '대구_기타 수상오락 서비스업': 23726.41221374046,
 '대구_기타 외국식 음식점업': 28466.44056057827,
 '대구_기타 주점업': 79331.8587366132,
 '대구_기타음식료품위주종합소매업': 25902.868325500393,
 '대구_내항 여객 운송업': 0.0,
 '대구_마사지업': 85949.92612819777,
 '대구_면세점': 74998.00387011515,
 '대구_버스 운송업': 12394.720301697047,
 '대구_비알콜 음료점업': 7566.209591137868,
 '대구_빵 및 과자류 소매업': 10117.258418183961,
 '대구_서양식 음식점업': 13840.145736474411,
 '대구_수산물 소매업': 39359.66309240824,
 '대구_슈퍼마켓': 14326.80227148483,
 '대구_스포츠 및 레크레이션 용품 임대업': 90797.34063923448,
 '대구_여관업': 39875.479337767785,
 '대구_여행사업': 635351.724137931,
 '대구_욕탕업': 16473.642167546073,
 '대구_육류 소매업': 28139.02537032847,
 '대구_일반유흥 주점업': 74482.53261856931,
 '대구_일식 음식점업': 38378.527934495934,
 '대구_자동차 임대업': 124103.8382838284,
 '대구_전시 및 행사 대행업': 33944.7450175127,
 '대구_정기 항공 운송업': 0.0,
 '대구_중식 음식점업': 21157.512446131946,
 '대구_차량용 가스 충전업': 34340.55456997977,
 '대구_차량용 주유소 운영업': 51474.41122060891,
 '대구_체인화 편의점': 6548.730940555163,
 '대구_택시 운송업': 6833.334960629263,
 '대구_피자 햄버거 샌드위치 및 유사 음식점업': 9197.671087992661,
 '대구_한식 음식점업': 31040.90912255577,
 '대구_호텔업': 148931.67672155047,
 '대구_화장품 및 방향제 소매업': 32798.99948725843,
 '대구_휴양콘도 운영업': 0.0,
 '대전_건강보조식품 소매업': 92358.6796442288,
 '대전_골프장 운영업': 47267.462190542865,
 '대전_과실 및 채소 소매업': 26530.3121276592,
 '대전_관광 민예품 및 선물용품 소매업': 38442.212644541214,
 '대전_그외 기타 분류안된 오락관련 서비스업': 0.0,
 '대전_그외 기타 스포츠시설 운영업': 26895.820433436533,
 '대전_그외 기타 종합 소매업': 13560.950043290888,
 '대전_기타 대형 종합 소매업': 31694.048299762962,
 '대전_기타 수상오락 서비스업': 34069.5238095238,
 '대전_기타 외국식 음식점업': 24490.63890371076,
 '대전_기타 주점업': 101111.12716433613,
 '대전_기타음식료품위주종합소매업': 23071.717298038882,
 '대전_내항 여객 운송업': 0.0,
 '대전_마사지업': 108639.88330781012,
 '대전_면세점': 0.0,
 '대전_버스 운송업': 0.0,
 '대전_비알콜 음료점업': 7215.005972827746,
 '대전_빵 및 과자류 소매업': 10763.389415418455,
 '대전_서양식 음식점업': 12624.725738788773,
 '대전_수산물 소매업': 39427.38573257582,
 '대전_슈퍼마켓': 13893.525849003534,
 '대전_스포츠 및 레크레이션 용품 임대업': 89713.83422125204,
 '대전_여관업': 36260.48380549632,
 '대전_여행사업': 1515000.0,
 '대전_욕탕업': 14898.423402611363,
 '대전_육류 소매업': 32175.529502690282,
 '대전_일반유흥 주점업': 115953.84931574285,
 '대전_일식 음식점업': 42184.365946873375,
 '대전_자동차 임대업': 133211.54335260118,
 '대전_전시 및 행사 대행업': 25500.41839065019,
 '대전_정기 항공 운송업': 0.0,
 '대전_중식 음식점업': 22322.727703966335,
 '대전_차량용 가스 충전업': 35021.228541344564,
 '대전_차량용 주유소 운영업': 49240.46350441467,
 '대전_체인화 편의점': 6586.412427271744,
 '대전_택시 운송업': 0.0,
 '대전_피자 햄버거 샌드위치 및 유사 음식점업': 9237.320366253214,
 '대전_한식 음식점업': 29526.268393165625,
 '대전_호텔업': 37373.82428257052,
 '대전_화장품 및 방향제 소매업': 35618.344258816775,
 '대전_휴양콘도 운영업': 0.0,
 '부산_건강보조식품 소매업': 96075.77705273927,
 '부산_골프장 운영업': 112325.90601094604,
 '부산_과실 및 채소 소매업': 18005.10024786872,
 '부산_관광 민예품 및 선물용품 소매업': 40672.00097344444,
 '부산_그외 기타 분류안된 오락관련 서비스업': 0.0,
 '부산_그외 기타 스포츠시설 운영업': 17137.46439975835,
 '부산_그외 기타 종합 소매업': 30271.724027072585,
 '부산_기타 대형 종합 소매업': 31591.738955958685,
 '부산_기타 수상오락 서비스업': 8576.51843183148,
 '부산_기타 외국식 음식점업': 24005.041409426107,
 '부산_기타 주점업': 88334.2453187482,
 '부산_기타음식료품위주종합소매업': 25097.125232679453,
 '부산_내항 여객 운송업': 60598.42180774749,
 '부산_마사지업': 89324.73439789041,
 '부산_면세점': 107009.0032134904,
 '부산_버스 운송업': 9310.73379244835,
 '부산_비알콜 음료점업': 6303.288352144934,
 '부산_빵 및 과자류 소매업': 10406.753647402684,
 '부산_서양식 음식점업': 15097.291860421876,
 '부산_수산물 소매업': 62419.81565476274,
 '부산_슈퍼마켓': 13226.668606160945,
 '부산_스포츠 및 레크레이션 용품 임대업': 93093.61169060756,
 '부산_여관업': 43351.947030532894,
 '부산_여행사업': 59198.69940438474,
 '부산_욕탕업': 19802.543854978896,
 '부산_육류 소매업': 32541.660575696147,
 '부산_일반유흥 주점업': 96281.99624872264,
 '부산_일식 음식점업': 47443.350980925556,
 '부산_자동차 임대업': 120705.9982300885,
 '부산_전시 및 행사 대행업': 19177.211952087328,
 '부산_정기 항공 운송업': 134238.8884913687,
 '부산_중식 음식점업': 23389.019449024192,
 '부산_차량용 가스 충전업': 32965.44332028033,
 '부산_차량용 주유소 운영업': 53119.68776834822,
 '부산_체인화 편의점': 6444.8798738599,
 '부산_택시 운송업': 6652.5253068860775,
 '부산_피자 햄버거 샌드위치 및 유사 음식점업': 9384.494148071275,
 '부산_한식 음식점업': 32430.911873712455,
 '부산_호텔업': 69633.4729664003,
 '부산_화장품 및 방향제 소매업': 32820.80129349465,
 '부산_휴양콘도 운영업': 68297.61406096986,
 '서울_건강보조식품 소매업': 80467.53742925901,
 '서울_골프장 운영업': 82253.79150864859,
 '서울_과실 및 채소 소매업': 22147.25112559503,
 '서울_관광 민예품 및 선물용품 소매업': 19950.235256100474,
 '서울_그외 기타 분류안된 오락관련 서비스업': 0.0,
 '서울_그외 기타 스포츠시설 운영업': 24537.539636760463,
 '서울_그외 기타 종합 소매업': 64722.63893754919,
 '서울_기타 대형 종합 소매업': 27876.1243967898,
 '서울_기타 수상오락 서비스업': 26406.525687597303,
 '서울_기타 외국식 음식점업': 19035.067640553483,
 '서울_기타 주점업': 96359.11185698093,
 '서울_기타음식료품위주종합소매업': 9525.836276519485,
 '서울_내항 여객 운송업': 39398.23862419666,
 '서울_마사지업': 105519.37921615201,
 '서울_면세점': 149379.91672676455,
 '서울_버스 운송업': 11728.467570071167,
 '서울_비알콜 음료점업': 6749.944770326597,
 '서울_빵 및 과자류 소매업': 8645.620135074076,
 '서울_서양식 음식점업': 15120.650062568628,
 '서울_수산물 소매업': 32991.27306073869,
 '서울_슈퍼마켓': 14119.33598114114,
 '서울_스포츠 및 레크레이션 용품 임대업': 85887.0709314037,
 '서울_여관업': 46994.037763023014,
 '서울_여행사업': 657835.8953455677,
 '서울_욕탕업': 13831.595811206222,
 '서울_육류 소매업': 28795.738175591297,
 '서울_일반유흥 주점업': 92156.71935385412,
 '서울_일식 음식점업': 35806.18752793891,
 '서울_자동차 임대업': 162225.9520710059,
 '서울_전시 및 행사 대행업': 22588.98053207588,
 '서울_정기 항공 운송업': 285609.4691633186,
 '서울_중식 음식점업': 23846.00286135986,
 '서울_차량용 가스 충전업': 30747.67967919673,
 '서울_차량용 주유소 운영업': 43765.25484351772,
 '서울_체인화 편의점': 5973.186700015488,
 '서울_택시 운송업': 7583.501157808942,
 '서울_피자 햄버거 샌드위치 및 유사 음식점업': 9302.466405551117,
 '서울_한식 음식점업': 27161.852103310444,
 '서울_호텔업': 55723.44779561694,
 '서울_화장품 및 방향제 소매업': 57739.9701311463,
 '서울_휴양콘도 운영업': 0.0,
 '세종_건강보조식품 소매업': 143844.77818853973,
 '세종_골프장 운영업': 143259.36303406543,
 '세종_과실 및 채소 소매업': 24203.140726581903,
 '세종_관광 민예품 및 선물용품 소매업': 13110.714285714286,
 '세종_그외 기타 분류안된 오락관련 서비스업': 39333.33333333333,
 '세종_그외 기타 스포츠시설 운영업': 56000.0,
 '세종_그외 기타 종합 소매업': 48201.751583391975,
 '세종_기타 대형 종합 소매업': 27185.57822299887,
 '세종_기타 수상오락 서비스업': 22733.333333333332,
 '세종_기타 외국식 음식점업': 24955.4766434797,
 '세종_기타 주점업': 153853.25365205843,
 '세종_기타음식료품위주종합소매업': 25541.183009340333,
 '세종_내항 여객 운송업': 0.0,
 '세종_마사지업': 110654.15129151291,
 '세종_면세점': 0.0,
 '세종_버스 운송업': 0.0,
 '세종_비알콜 음료점업': 8000.822333066784,
 '세종_빵 및 과자류 소매업': 9673.170506106364,
 '세종_서양식 음식점업': 14394.81040863747,
 '세종_수산물 소매업': 43170.950591243214,
 '세종_슈퍼마켓': 15094.521969687768,
 '세종_스포츠 및 레크레이션 용품 임대업': 73602.45868854373,
 '세종_여관업': 41555.225076186325,
 '세종_여행사업': 0.0,
 '세종_욕탕업': 11760.676679473745,
 '세종_육류 소매업': 28790.68083728171,
 '세종_일반유흥 주점업': 185244.2842178771,
 '세종_일식 음식점업': 38315.36039277775,
 '세종_자동차 임대업': 0.0,
 '세종_전시 및 행사 대행업': 17294.423444323143,
 '세종_정기 항공 운송업': 0.0,
 '세종_중식 음식점업': 24565.68139131725,
 '세종_차량용 가스 충전업': 33890.78134452141,
 '세종_차량용 주유소 운영업': 56571.27999555514,
 '세종_체인화 편의점': 6959.677211154617,
 '세종_택시 운송업': 0.0,
 '세종_피자 햄버거 샌드위치 및 유사 음식점업': 10451.176748759352,
 '세종_한식 음식점업': 28981.59888377375,
 '세종_호텔업': 0.0,
 '세종_화장품 및 방향제 소매업': 32576.09840397789,
 '세종_휴양콘도 운영업': 0.0,
 '울산_건강보조식품 소매업': 139814.14269141533,
 '울산_골프장 운영업': 103883.21198596677,
 '울산_과실 및 채소 소매업': 38648.99568570082,
 '울산_관광 민예품 및 선물용품 소매업': 28595.061004223367,
 '울산_그외 기타 분류안된 오락관련 서비스업': 45000.0,
 '울산_그외 기타 스포츠시설 운영업': 19452.510558423273,
 '울산_그외 기타 종합 소매업': 40244.01546698393,
 '울산_기타 대형 종합 소매업': 34857.32027057504,
 '울산_기타 수상오락 서비스업': 38348.641304347824,
 '울산_기타 외국식 음식점업': 36291.50480441227,
 '울산_기타 주점업': 89973.77676976525,
 '울산_기타음식료품위주종합소매업': 31057.859373440762,
 '울산_내항 여객 운송업': 0.0,
 '울산_마사지업': 133126.8029395754,
 '울산_면세점': 225238.08510638296,
 '울산_버스 운송업': 13623.241817442567,
 '울산_비알콜 음료점업': 8073.296213755235,
 '울산_빵 및 과자류 소매업': 11032.695164030229,
 '울산_서양식 음식점업': 17573.648571841273,
 '울산_수산물 소매업': 52963.39601910267,
 '울산_슈퍼마켓': 14395.633744410236,
 '울산_스포츠 및 레크레이션 용품 임대업': 79545.54109367338,
 '울산_여관업': 42236.59713829871,
 '울산_여행사업': 18467.88990825688,
 '울산_욕탕업': 20437.646550118905,
 '울산_육류 소매업': 36187.09339037473,
 '울산_일반유흥 주점업': 118270.54957557026,
 '울산_일식 음식점업': 50135.65532732723,
 '울산_자동차 임대업': 177023.20512820513,
 '울산_전시 및 행사 대행업': 34892.00910375137,
 '울산_정기 항공 운송업': 0.0,
 '울산_중식 음식점업': 23493.624673159167,
 '울산_차량용 가스 충전업': 33586.392073496856,
 '울산_차량용 주유소 운영업': 55443.29408426224,
 '울산_체인화 편의점': 7201.730750570549,
 '울산_택시 운송업': 5948.484848484848,
 '울산_피자 햄버거 샌드위치 및 유사 음식점업': 10351.978556043436,
 '울산_한식 음식점업': 35110.26706975821,
 '울산_호텔업': 14226.626693738364,
 '울산_화장품 및 방향제 소매업': 40516.618826745165,
 '울산_휴양콘도 운영업': 0.0,
 '인천_건강보조식품 소매업': 103091.56177804465,
 '인천_골프장 운영업': 151151.54146118445,
 '인천_과실 및 채소 소매업': 29188.0427279468,
 '인천_관광 민예품 및 선물용품 소매업': 41524.96758199848,
 '인천_그외 기타 분류안된 오락관련 서비스업': 0.0,
 '인천_그외 기타 스포츠시설 운영업': 15553.715564472566,
 '인천_그외 기타 종합 소매업': 8939.971509420408,
 '인천_기타 대형 종합 소매업': 29622.456322591326,
 '인천_기타 수상오락 서비스업': 37655.90783410138,
 '인천_기타 외국식 음식점업': 21471.842106757216,
 '인천_기타 주점업': 95197.90427364282,
 '인천_기타음식료품위주종합소매업': 26994.32427264776,
 '인천_내항 여객 운송업': 42149.161876295526,
 '인천_마사지업': 88667.83729966004,
 '인천_면세점': 109348.30507352001,
 '인천_버스 운송업': 0.0,
 '인천_비알콜 음료점업': 6653.774665697845,
 '인천_빵 및 과자류 소매업': 9933.902316178299,
 '인천_서양식 음식점업': 15401.096474695418,
 '인천_수산물 소매업': 42406.49586154852,
 '인천_슈퍼마켓': 16152.798906160488,
 '인천_스포츠 및 레크레이션 용품 임대업': 79348.89021093659,
 '인천_여관업': 40947.66966824196,
 '인천_여행사업': 27102.717391304348,
 '인천_욕탕업': 13564.857990641472,
 '인천_육류 소매업': 26904.07085311921,
 '인천_일반유흥 주점업': 83509.1764640018,
 '인천_일식 음식점업': 49253.91451388454,
 '인천_자동차 임대업': 200620.90721649487,
 '인천_전시 및 행사 대행업': 29526.96634186946,
 '인천_정기 항공 운송업': 227838.23685576822,
 '인천_중식 음식점업': 22390.269694894152,
 '인천_차량용 가스 충전업': 30562.097711075814,
 '인천_차량용 주유소 운영업': 52804.4142646279,
 '인천_체인화 편의점': 6971.01728075598,
 '인천_택시 운송업': 7058.795133172311,
 '인천_피자 햄버거 샌드위치 및 유사 음식점업': 10087.82414230819,
 '인천_한식 음식점업': 30912.88536753885,
 '인천_호텔업': 94774.04362591846,
 '인천_화장품 및 방향제 소매업': 35058.23522601367,
 '인천_휴양콘도 운영업': 0.0,
 '전남_건강보조식품 소매업': 143092.24170356733,
 '전남_골프장 운영업': 103648.35869525498,
 '전남_과실 및 채소 소매업': 42712.62478289705,
 '전남_관광 민예품 및 선물용품 소매업': 26772.153423772612,
 '전남_그외 기타 분류안된 오락관련 서비스업': 29818.18181818182,
 '전남_그외 기타 스포츠시설 운영업': 75696.23211875843,
 '전남_그외 기타 종합 소매업': 33111.05471568101,
 '전남_기타 대형 종합 소매업': 41036.031828947984,
 '전남_기타 수상오락 서비스업': 99382.35294117648,
 '전남_기타 외국식 음식점업': 29900.408647263794,
 '전남_기타 주점업': 102998.30286503014,
 '전남_기타음식료품위주종합소매업': 32346.46219750534,
 '전남_내항 여객 운송업': 21768.95494048054,
 '전남_마사지업': 111960.5767210589,
 '전남_면세점': 59427.81840454724,
 '전남_버스 운송업': 7362.536829301257,
 '전남_비알콜 음료점업': 8612.621835875463,
 '전남_빵 및 과자류 소매업': 13024.29589314365,
 '전남_서양식 음식점업': 15472.963284507283,
 '전남_수산물 소매업': 67647.25240484704,
 '전남_슈퍼마켓': 17508.968963243453,
 '전남_스포츠 및 레크레이션 용품 임대업': 92742.79413084875,
 '전남_여관업': 49763.57504355833,
 '전남_여행사업': 13776.027940943008,
 '전남_욕탕업': 17153.57223792339,
 '전남_육류 소매업': 46323.139104261696,
 '전남_일반유흥 주점업': 168828.7171216864,
 '전남_일식 음식점업': 60496.20869295877,
 '전남_자동차 임대업': 178126.81104033967,
 '전남_전시 및 행사 대행업': 25726.881423401246,
 '전남_정기 항공 운송업': 0.0,
 '전남_중식 음식점업': 25231.416761360815,
 '전남_차량용 가스 충전업': 35805.60679914094,
 '전남_차량용 주유소 운영업': 69481.71605961365,
 '전남_체인화 편의점': 8195.519528145433,
 '전남_택시 운송업': 6070.0,
 '전남_피자 햄버거 샌드위치 및 유사 음식점업': 12189.911573877916,
 '전남_한식 음식점업': 36495.214297623534,
 '전남_호텔업': 75034.08908594816,
 '전남_화장품 및 방향제 소매업': 43483.50542167196,
 '전남_휴양콘도 운영업': 57386.39742510718,
 '전북_건강보조식품 소매업': 102413.21576774397,
 '전북_골프장 운영업': 101092.58176327018,
 '전북_과실 및 채소 소매업': 27893.415113325398,
 '전북_관광 민예품 및 선물용품 소매업': 28177.36295960642,
 '전북_그외 기타 분류안된 오락관련 서비스업': 0.0,
 '전북_그외 기타 스포츠시설 운영업': 13978.02780638517,
 '전북_그외 기타 종합 소매업': 46156.75444122732,
 '전북_기타 대형 종합 소매업': 35835.655359217504,
 '전북_기타 수상오락 서비스업': 40275.289017341034,
 '전북_기타 외국식 음식점업': 25082.343674425712,
 '전북_기타 주점업': 98370.96576250048,
 '전북_기타음식료품위주종합소매업': 27862.890017312555,
 '전북_내항 여객 운송업': 105968.33333333334,
 '전북_마사지업': 115625.47343313045,
 '전북_면세점': 0.0,
 '전북_버스 운송업': 33794.840000000004,
 '전북_비알콜 음료점업': 7381.832252921435,
 '전북_빵 및 과자류 소매업': 12084.97717025284,
 '전북_서양식 음식점업': 15432.82049382958,
 '전북_수산물 소매업': 57587.75162791001,
 '전북_슈퍼마켓': 16119.154860245104,
 '전북_스포츠 및 레크레이션 용품 임대업': 99421.72727204343,
 '전북_여관업': 43559.66519540498,
 '전북_여행사업': 111909.36507936507,
 '전북_욕탕업': 17619.187273528085,
 '전북_육류 소매업': 38099.94244574083,
 '전북_일반유흥 주점업': 160585.6084567374,
 '전북_일식 음식점업': 52711.8239611407,
 '전북_자동차 임대업': 170043.01361867704,
 '전북_전시 및 행사 대행업': 10712.468300819964,
 '전북_정기 항공 운송업': 136523.02273028903,
 '전북_중식 음식점업': 23998.12969838696,
 '전북_차량용 가스 충전업': 36438.28888523205,
 '전북_차량용 주유소 운영업': 65802.7150277543,
 '전북_체인화 편의점': 7207.667761340672,
 '전북_택시 운송업': 18896.41791044776,
 '전북_피자 햄버거 샌드위치 및 유사 음식점업': 10826.00681546237,
 '전북_한식 음식점업': 32706.652838367558,
 '전북_호텔업': 68231.5909090909,
 '전북_화장품 및 방향제 소매업': 38068.56280711028,
 '전북_휴양콘도 운영업': 68627.05208097654,
 '제주_건강보조식품 소매업': 150972.7881473276,
 '제주_골프장 운영업': 110996.69930739974,
 '제주_과실 및 채소 소매업': 51013.57753489046,
 '제주_관광 민예품 및 선물용품 소매업': 30018.44650405633,
 '제주_그외 기타 분류안된 오락관련 서비스업': 14730.462585034013,
 '제주_그외 기타 스포츠시설 운영업': 14670.93504206613,
 '제주_그외 기타 종합 소매업': 59611.46909067059,
 '제주_기타 대형 종합 소매업': 47503.20915980318,
 '제주_기타 수상오락 서비스업': 44869.207317073175,
 '제주_기타 외국식 음식점업': 30786.667866373282,
 '제주_기타 주점업': 183409.96198938016,
 '제주_기타음식료품위주종합소매업': 36268.73026470616,
 '제주_내항 여객 운송업': 29785.195733823897,
 '제주_마사지업': 125675.92592592593,
 '제주_면세점': 68933.07362518768,
 '제주_버스 운송업': 0.0,
 '제주_비알콜 음료점업': 8591.99138107499,
 '제주_빵 및 과자류 소매업': 13020.06401821276,
 '제주_서양식 음식점업': 20524.213552982183,
 '제주_수산물 소매업': 60871.21244717783,
 '제주_슈퍼마켓': 23572.520652937055,
 '제주_스포츠 및 레크레이션 용품 임대업': 101061.59634807531,
 '제주_여관업': 59807.772216645455,
 '제주_여행사업': 66401.19849494201,
 '제주_욕탕업': 22688.928338694182,
 '제주_육류 소매업': 56039.85874941042,
 '제주_일반유흥 주점업': 182751.10248735387,
 '제주_일식 음식점업': 55383.6387571816,
 '제주_자동차 임대업': 67300.95482759517,
 '제주_전시 및 행사 대행업': 24946.98500964031,
 '제주_정기 항공 운송업': 151665.86669006268,
 '제주_중식 음식점업': 27060.75493168834,
 '제주_차량용 가스 충전업': 54893.49639177467,
 '제주_차량용 주유소 운영업': 52909.848351789136,
 '제주_체인화 편의점': 7797.65964759146,
 '제주_택시 운송업': 6016.727272727272,
 '제주_피자 햄버거 샌드위치 및 유사 음식점업': 11413.43352658998,
 '제주_한식 음식점업': 37591.67809984937,
 '제주_호텔업': 218672.7776679588,
 '제주_화장품 및 방향제 소매업': 48540.681895994836,
 '제주_휴양콘도 운영업': 69140.56960784314,
 '충남_건강보조식품 소매업': 86582.91177786578,
 '충남_골프장 운영업': 104043.70332598448,
 '충남_과실 및 채소 소매업': 31593.468500100666,
 '충남_관광 민예품 및 선물용품 소매업': 22212.28092573997,
 '충남_그외 기타 분류안된 오락관련 서비스업': 43071.232876712325,
 '충남_그외 기타 스포츠시설 운영업': 19020.0284729507,
 '충남_그외 기타 종합 소매업': 33831.70101654486,
 '충남_기타 대형 종합 소매업': 31963.604305670597,
 '충남_기타 수상오락 서비스업': 41953.42481598318,
 '충남_기타 외국식 음식점업': 27214.23830371079,
 '충남_기타 주점업': 114473.5352304081,
 '충남_기타음식료품위주종합소매업': 29480.000994305272,
 '충남_내항 여객 운송업': 32004.94252873563,
 '충남_마사지업': 96904.77181521231,
 '충남_면세점': 0.0,
 '충남_버스 운송업': 3943.2441651705567,
 '충남_비알콜 음료점업': 8138.202424479191,
 '충남_빵 및 과자류 소매업': 11872.712476525894,
 '충남_서양식 음식점업': 13084.502816889375,
 '충남_수산물 소매업': 55162.33247612146,
 '충남_슈퍼마켓': 17950.336613536034,
 '충남_스포츠 및 레크레이션 용품 임대업': 74706.28969877702,
 '충남_여관업': 41969.21553038606,
 '충남_여행사업': 17040.7205519121,
 '충남_욕탕업': 17176.650709770383,
 '충남_육류 소매업': 34857.6816988461,
 '충남_일반유흥 주점업': 138895.20857073585,
 '충남_일식 음식점업': 55032.63685892289,
 '충남_자동차 임대업': 172365.2408759124,
 '충남_전시 및 행사 대행업': 12355.003775499545,
 '충남_정기 항공 운송업': 193223.1182795699,
 '충남_중식 음식점업': 24451.528510315507,
 '충남_차량용 가스 충전업': 37303.214634683216,
 '충남_차량용 주유소 운영업': 65987.71486165047,
 '충남_체인화 편의점': 7535.94548289751,
 '충남_택시 운송업': 8362.797202797203,
 '충남_피자 햄버거 샌드위치 및 유사 음식점업': 11092.06541504478,
 '충남_한식 음식점업': 27413.848705665245,
 '충남_호텔업': 38284.5544652813,
 '충남_화장품 및 방향제 소매업': 63569.78882917964,
 '충남_휴양콘도 운영업': 59026.040517114736,
 '충북_건강보조식품 소매업': 121766.11914708442,
 '충북_골프장 운영업': 139073.97001431612,
 '충북_과실 및 채소 소매업': 26742.65100940934,
 '충북_관광 민예품 및 선물용품 소매업': 27596.92370661428,
 '충북_그외 기타 분류안된 오락관련 서비스업': 0.0,
 '충북_그외 기타 스포츠시설 운영업': 18146.105963411737,
 '충북_그외 기타 종합 소매업': 35870.75729357408,
 '충북_기타 대형 종합 소매업': 32749.93320303552,
 '충북_기타 수상오락 서비스업': 23689.97335811648,
 '충북_기타 외국식 음식점업': 28344.137416693593,
 '충북_기타 주점업': 114296.6613785759,
 '충북_기타음식료품위주종합소매업': 29725.095775488793,
 '충북_내항 여객 운송업': 35904.29234338747,
 '충북_마사지업': 105749.69400630916,
 '충북_면세점': 81040.64122137404,
 '충북_버스 운송업': 10065.217391304348,
 '충북_비알콜 음료점업': 8101.404490967725,
 '충북_빵 및 과자류 소매업': 11107.299440625806,
 '충북_서양식 음식점업': 15263.406467403369,
 '충북_수산물 소매업': 37359.7648111294,
 '충북_슈퍼마켓': 17716.38494002297,
 '충북_스포츠 및 레크레이션 용품 임대업': 92230.12460888534,
 '충북_여관업': 40112.03629095541,
 '충북_여행사업': 61562.21374045801,
 '충북_욕탕업': 13796.653505548109,
 '충북_육류 소매업': 35671.94550523006,
 '충북_일반유흥 주점업': 161711.4241121879,
 '충북_일식 음식점업': 54369.4308461774,
 '충북_자동차 임대업': 125887.47321428571,
 '충북_전시 및 행사 대행업': 22742.397280092595,
 '충북_정기 항공 운송업': 0.0,
 '충북_중식 음식점업': 24910.298315765707,
 '충북_차량용 가스 충전업': 45461.764092580015,
 '충북_차량용 주유소 운영업': 63556.98335347905,
 '충북_체인화 편의점': 7693.229158366719,
 '충북_택시 운송업': 6206.10248447205,
 '충북_피자 햄버거 샌드위치 및 유사 음식점업': 10653.98371384955,
 '충북_한식 음식점업': 28440.324097187582,
 '충북_호텔업': 45434.79968322774,
 '충북_화장품 및 방향제 소매업': 41068.81476361743,
 '충북_휴양콘도 운영업': 59252.16689887805}

In [ ]:
df_power = pd.DataFrame(dic_power.items(), columns=["CLSS", "POWER"])
df_power["CARD_SIDO_NM"] = df_power["CLSS"].apply(lambda x: x.split("_")[0])
df_power["STD_CLSS_NM"] = df_power["CLSS"].apply(lambda x: x.split("_")[1])
df_power.drop("CLSS", axis=1, inplace=True)
df_power
# 나중에 merge 예정

,POWER,CARD_SIDO_NM,STD_CLSS_NM
0,112121.205378,강원,건강보조식품 소매업
1,112546.795906,강원,골프장 운영업
2,38558.158795,강원,과실 및 채소 소매업
3,20761.327287,강원,관광 민예품 및 선물용품 소매업
4,17700.000000,강원,그외 기타 분류안된 오락관련 서비스업
...,...,...,...
692,10653.983714,충북,피자 햄버거 샌드위치 및 유사 음식점업
693,28440.324097,충북,한식 음식점업
694,45434.799683,충북,호텔업
695,41068.814764,충북,화장품 및 방향제 소매업


## 계절 사용량
-> 별로 쓸모가 없음

In [ ]:
def season(series):
  spring = [3, 4, 5]
  summer = [6, 7, 8]
  autumn = [9, 10, 11]
  winter = [12, 1, 2]
  if series % 100 in winter:
    return 3
  elif series % 100 in spring:
    return 0
  elif series % 100 in summer:
    return 1
  elif series % 100 in autumn:
    return 2

## 코로나 적용(3월 이상 적용)

In [ ]:
def covid(series):
  if ((series // 100) == 2020) & ((series % 100) > 2):
    return 1
  else:
    return 0

## 외지인 비율

In [ ]:
df_non = df[df['CARD_SIDO_NM'] != df['HOM_SIDO_NM']]

In [ ]:
group_non = df_non.groupby(by = ['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])
del df_non

In [ ]:
group_non = group_non.count()['AMT']

In [ ]:
group = df.groupby(by = ['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])

In [ ]:
group = group.count()['AMT']

In [ ]:
tourist_rate = np.round((group_non / group) * 100, 1)

In [ ]:
del group_non

In [ ]:
tourist_rate = tourist_rate.reset_index()

In [ ]:
tourist_rate = tourist_rate.rename(columns = {'AMT':'tourist_rate'})

In [ ]:
tourist_rate

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,tourist_rate
0,201901,강원,건강보조식품 소매업,5.3
1,201901,강원,골프장 운영업,90.9
2,201901,강원,과실 및 채소 소매업,77.2
3,201901,강원,관광 민예품 및 선물용품 소매업,77.6
4,201901,강원,그외 기타 스포츠시설 운영업,NaN
...,...,...,...,...
10040,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,46.0
10041,202004,충북,한식 음식점업,82.8
10042,202004,충북,호텔업,39.3
10043,202004,충북,화장품 및 방향제 소매업,10.2


## 전처리 적용

In [ ]:
group = df.groupby(by = ['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])
group = group.sum()['AMT']
group = group.reset_index()
group

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500
...,...,...,...,...
10040,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928
10041,202004,충북,한식 음식점업,18911036160
10042,202004,충북,호텔업,14121500
10043,202004,충북,화장품 및 방향제 소매업,450507431


In [ ]:
df_sub = df_submission.copy()

In [ ]:
group = pd.merge(group, df_power, on=["STD_CLSS_NM", "CARD_SIDO_NM"])
group = group.sort_values(by = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'] )  # 정렬 기준 순서대로 정렬
group = group.reset_index().drop(['index'], axis = 1); group

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER
0,201901,강원,건강보조식품 소매업,148354746,112121.205378
1,201901,강원,골프장 운영업,1229200312,112546.795906
2,201901,강원,과실 및 채소 소매업,1286979106,38558.158795
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770,20761.327287
4,201901,강원,그외 기타 스포츠시설 운영업,13101500,58204.367816
...,...,...,...,...,...
10040,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928,10653.983714
10041,202004,충북,한식 음식점업,18911036160,28440.324097
10042,202004,충북,호텔업,14121500,45434.799683
10043,202004,충북,화장품 및 방향제 소매업,450507431,41068.814764


In [ ]:
df_sub = pd.merge(df_sub, df_power, on=["STD_CLSS_NM", "CARD_SIDO_NM"])
df_sub = df_sub.sort_values(by = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'] )  # 정렬 기준 순서대로 정렬
df_sub = df_sub.reset_index().drop(['index'], axis = 1); df_sub

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER
0,202004,강원,건강보조식품 소매업,0,112121.205378
1,202004,강원,골프장 운영업,0,112546.795906
2,202004,강원,과실 및 채소 소매업,0,38558.158795
3,202004,강원,관광 민예품 및 선물용품 소매업,0,20761.327287
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0,17700.000000
...,...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0,10653.983714
1390,202007,충북,한식 음식점업,0,28440.324097
1391,202007,충북,호텔업,0,45434.799683
1392,202007,충북,화장품 및 방향제 소매업,0,41068.814764


In [ ]:
# COVID 적용
group['COVID19'] = group['REG_YYMM'].map(covid)
df_sub['COVID19'] = df_sub['REG_YYMM'].map(covid)

In [ ]:
df_sub

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER,COVID19
0,202004,강원,건강보조식품 소매업,0,112121.205378,1
1,202004,강원,골프장 운영업,0,112546.795906,1
2,202004,강원,과실 및 채소 소매업,0,38558.158795,1
3,202004,강원,관광 민예품 및 선물용품 소매업,0,20761.327287,1
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0,17700.000000,1
...,...,...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0,10653.983714,1
1390,202007,충북,한식 음식점업,0,28440.324097,1
1391,202007,충북,호텔업,0,45434.799683,1
1392,202007,충북,화장품 및 방향제 소매업,0,41068.814764,1


In [ ]:
# 외지인 비율
tourist_rate = tourist_rate.fillna(0)
group = pd.merge(group, tourist_rate, on=['REG_YYMM', "STD_CLSS_NM", "CARD_SIDO_NM"])
df_sub = pd.merge(df_sub, tourist_rate, on=['REG_YYMM' , "STD_CLSS_NM", "CARD_SIDO_NM"], how = 'left')

In [ ]:
df_sub

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER,COVID19,tourist_rate
0,202004,강원,건강보조식품 소매업,0,112121.205378,1,2.9
1,202004,강원,골프장 운영업,0,112546.795906,1,85.4
2,202004,강원,과실 및 채소 소매업,0,38558.158795,1,68.3
3,202004,강원,관광 민예품 및 선물용품 소매업,0,20761.327287,1,56.2
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0,17700.000000,1,NaN
...,...,...,...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0,10653.983714,1,NaN
1390,202007,충북,한식 음식점업,0,28440.324097,1,NaN
1391,202007,충북,호텔업,0,45434.799683,1,NaN
1392,202007,충북,화장품 및 방향제 소매업,0,41068.814764,1,NaN


In [ ]:
na_list = df_sub[(df_sub['REG_YYMM'] == 202004) & df_sub['tourist_rate'].isna()].index

In [ ]:
df_sub.iloc[na_list, 6] = 0

In [ ]:
df_sub

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER,COVID19,tourist_rate
0,202004,강원,건강보조식품 소매업,0,112121.205378,1,2.9
1,202004,강원,골프장 운영업,0,112546.795906,1,85.4
2,202004,강원,과실 및 채소 소매업,0,38558.158795,1,68.3
3,202004,강원,관광 민예품 및 선물용품 소매업,0,20761.327287,1,56.2
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0,17700.000000,1,0.0
...,...,...,...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0,10653.983714,1,NaN
1390,202007,충북,한식 음식점업,0,28440.324097,1,NaN
1391,202007,충북,호텔업,0,45434.799683,1,NaN
1392,202007,충북,화장품 및 방향제 소매업,0,41068.814764,1,NaN


In [ ]:
group

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER,COVID19,tourist_rate
0,201901,강원,건강보조식품 소매업,148354746,112121.205378,0,5.3
1,201901,강원,골프장 운영업,1229200312,112546.795906,0,90.9
2,201901,강원,과실 및 채소 소매업,1286979106,38558.158795,0,77.2
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770,20761.327287,0,77.6
4,201901,강원,그외 기타 스포츠시설 운영업,13101500,58204.367816,0,0.0
...,...,...,...,...,...,...,...
10040,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928,10653.983714,1,46.0
10041,202004,충북,한식 음식점업,18911036160,28440.324097,1,82.8
10042,202004,충북,호텔업,14121500,45434.799683,1,39.3
10043,202004,충북,화장품 및 방향제 소매업,450507431,41068.814764,1,10.2


In [ ]:
# 감소율
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

y_train = group['tourist_rate']
X_train = group.drop(['tourist_rate'], axis = 1)

rf = RandomForestRegressor(random_state = 0, n_estimators = 10)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
encoder.fit(X_train['CARD_SIDO_NM'])
X_train['CARD_SIDO_NM'] = encoder.transform(X_train['CARD_SIDO_NM'])
encoder.fit(X_train['STD_CLSS_NM'])
X_train['STD_CLSS_NM'] = encoder.transform(X_train['STD_CLSS_NM'])

In [ ]:
import time
import numpy as np
start = time.time()
rf.fit(X_train, y_train)
print(np.round(time.time() - start, 2))

0.25


In [ ]:
temp = df_sub[df_sub['REG_YYMM'] == 202007].drop(['tourist_rate'], axis = 1)

In [ ]:
encoder.fit(temp['CARD_SIDO_NM'])
temp['CARD_SIDO_NM'] = encoder.transform(temp['CARD_SIDO_NM'])
encoder.fit(temp['STD_CLSS_NM'])
temp['STD_CLSS_NM'] = encoder.transform(temp['STD_CLSS_NM'])

In [ ]:
pred = rf.predict(temp)

In [ ]:
len(pred)

697

In [ ]:
na_index = df_sub[df_sub['REG_YYMM'] == 202007].drop(['tourist_rate'], axis = 1).index

In [ ]:
df_sub.iloc[na_index, 6] = pred; df_sub

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER,COVID19,tourist_rate
0,202004,강원,건강보조식품 소매업,0,112121.205378,1,2.9
1,202004,강원,골프장 운영업,0,112546.795906,1,85.4
2,202004,강원,과실 및 채소 소매업,0,38558.158795,1,68.3
3,202004,강원,관광 민예품 및 선물용품 소매업,0,20761.327287,1,56.2
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0,17700.000000,1,0.0
...,...,...,...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0,10653.983714,1,15.0
1390,202007,충북,한식 음식점업,0,28440.324097,1,15.0
1391,202007,충북,호텔업,0,45434.799683,1,10.0
1392,202007,충북,화장품 및 방향제 소매업,0,41068.814764,1,15.0


In [ ]:
  group

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER,COVID19,tourist_rate
0,201901,강원,건강보조식품 소매업,148354746,112121.205378,0,5.3
1,201901,강원,골프장 운영업,1229200312,112546.795906,0,90.9
2,201901,강원,과실 및 채소 소매업,1286979106,38558.158795,0,77.2
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770,20761.327287,0,77.6
4,201901,강원,그외 기타 스포츠시설 운영업,13101500,58204.367816,0,0.0
...,...,...,...,...,...,...,...
10040,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928,10653.983714,1,46.0
10041,202004,충북,한식 음식점업,18911036160,28440.324097,1,82.8
10042,202004,충북,호텔업,14121500,45434.799683,1,39.3
10043,202004,충북,화장품 및 방향제 소매업,450507431,41068.814764,1,10.2


## 로그 적용

In [ ]:
group['AMT'] = np.log1p(group['AMT']); group

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER,COVID19,tourist_rate
0,201901,강원,건강보조식품 소매업,18.815117,112121.205378,0,5.3
1,201901,강원,골프장 운영업,20.929630,112546.795906,0,90.9
2,201901,강원,과실 및 채소 소매업,20.975564,38558.158795,0,77.2
3,201901,강원,관광 민예품 및 선물용품 소매업,17.238364,20761.327287,0,77.6
4,201901,강원,그외 기타 스포츠시설 운영업,16.388237,58204.367816,0,0.0
...,...,...,...,...,...,...,...
10040,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,21.040727,10653.983714,1,46.0
10041,202004,충북,한식 음식점업,23.663012,28440.324097,1,82.8
10042,202004,충북,호텔업,16.463209,45434.799683,1,39.3
10043,202004,충북,화장품 및 방향제 소매업,19.925885,41068.814764,1,10.2


## 년, 월 분리

In [ ]:
df_sub

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,POWER,COVID19,tourist_rate
0,202004,강원,건강보조식품 소매업,0,112121.205378,1,2.9
1,202004,강원,골프장 운영업,0,112546.795906,1,85.4
2,202004,강원,과실 및 채소 소매업,0,38558.158795,1,68.3
3,202004,강원,관광 민예품 및 선물용품 소매업,0,20761.327287,1,56.2
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0,17700.000000,1,0.0
...,...,...,...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0,10653.983714,1,15.0
1390,202007,충북,한식 음식점업,0,28440.324097,1,15.0
1391,202007,충북,호텔업,0,45434.799683,1,10.0
1392,202007,충북,화장품 및 방향제 소매업,0,41068.814764,1,15.0


In [ ]:
group['REG_YYMM'] = group['REG_YYMM'].apply(str)
df_sub['REG_YYMM'] = df_sub['REG_YYMM'].apply(str)

In [ ]:
group['year'] = group['REG_YYMM'].apply(lambda x: x[:4])
group['month'] = group['REG_YYMM'].apply(lambda x: x[4:])
df_sub['year'] = df_sub['REG_YYMM'].apply(lambda x: x[:4])
df_sub['month'] = df_sub['REG_YYMM'].apply(lambda x: x[4:])

In [ ]:
group = group.drop(['REG_YYMM'], axis = 1)
df_sub = df_sub.drop(['REG_YYMM'], axis = 1)

# 인코딩

## 레이블 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(group['CARD_SIDO_NM'])
group['CARD_SIDO_NM'] = encoder.transform(group['CARD_SIDO_NM'])
encoder.fit(group['STD_CLSS_NM'])
group['STD_CLSS_NM'] = encoder.transform(group['STD_CLSS_NM'])
group['year'] = group['year'].apply(int)
group['month'] = group['month'].apply(int)

## 원핫 인코딩

In [ ]:
# 인코딩은 원핫인코딩으로 진행
group = pd.get_dummies(group)
df_sub = pd.get_dummies(df_sub)

In [ ]:
group

,AMT,POWER,COVID19,tourist_rate,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,CARD_SIDO_NM_경남,CARD_SIDO_NM_경북,CARD_SIDO_NM_광주,CARD_SIDO_NM_대구,CARD_SIDO_NM_대전,CARD_SIDO_NM_부산,CARD_SIDO_NM_서울,CARD_SIDO_NM_세종,CARD_SIDO_NM_울산,CARD_SIDO_NM_인천,CARD_SIDO_NM_전남,CARD_SIDO_NM_전북,CARD_SIDO_NM_제주,CARD_SIDO_NM_충남,CARD_SIDO_NM_충북,STD_CLSS_NM_건강보조식품 소매업,STD_CLSS_NM_골프장 운영업,STD_CLSS_NM_과실 및 채소 소매업,STD_CLSS_NM_관광 민예품 및 선물용품 소매업,STD_CLSS_NM_그외 기타 분류안된 오락관련 서비스업,STD_CLSS_NM_그외 기타 스포츠시설 운영업,STD_CLSS_NM_그외 기타 종합 소매업,STD_CLSS_NM_기타 대형 종합 소매업,STD_CLSS_NM_기타 수상오락 서비스업,STD_CLSS_NM_기타 외국식 음식점업,STD_CLSS_NM_기타 주점업,STD_CLSS_NM_기타음식료품위주종합소매업,STD_CLSS_NM_내항 여객 운송업,STD_CLSS_NM_마사지업,STD_CLSS_NM_면세점,STD_CLSS_NM_버스 운송업,STD_CLSS_NM_비알콜 음료점업,STD_CLSS_NM_빵 및 과자류 소매업,STD_CLSS_NM_서양식 음식점업,STD_CLSS_NM_수산물 소매업,STD_CLSS_NM_슈퍼마켓,STD_CLSS_NM_스포츠 및 레크레이션 용품 임대업,STD_CLSS_NM_여관업,STD_CLSS_NM_여행사업,STD_CLSS_NM_욕탕업,STD_CLSS_NM_육류 소매업,STD_CLSS_NM_일반유흥 주점업,STD_CLSS_NM_일식 음식점업,STD_CLSS_NM_자동차 임대업,STD_CLSS_NM_전시 및 행사 대행업,STD_CLSS_NM_정기 항공 운송업,STD_CLSS_NM_중식 음식점업,STD_CLSS_NM_차량용 가스 충전업,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업,year_2019,year_2020,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,18.815117,112121.205378,0,5.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,20.929630,112546.795906,0,90.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,20.975564,38558.158795,0,77.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
3,17.238364,20761.327287,0,77.6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
4,16.388237,58204.367816,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10040,21.040727,10653.983714,1,46.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
10041,23.663012,28440.324097,1,82.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
10042,16.463209,45434.799683,1,39.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
10043,19.925885,41068.814764,1,10.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0


# 다중공선성

In [ ]:
def vif_cal(df, index):
  from sklearn.linear_model import LinearRegression
  vars = df.shape[1]
  x_i = df.iloc[:, index]
  x_noti = df.iloc[:, np.arange(vars) != index]
  r_square = LinearRegression().fit(X = x_noti, y = x_i).score(X = x_noti, y = x_i)
  vif = 1/(1-r_square)
  return vif

In [ ]:
vif = pd.DataFrame()
vif

""


In [ ]:
vif['VIF'] = [vif_cal(group.drop(['AMT'], axis = 1), i) for i in range(group.drop(['AMT'], axis = 1).shape[1])]
vif['Feature'] = group.drop(['AMT'], axis = 1).columns
vif.head(5)

,VIF,Feature
0,1.011052,CARD_SIDO_NM
1,1.056486,STD_CLSS_NM
2,1.030511,POWER
3,1.784716,COVID19
4,1.104552,tourist_rate


# 예측 모델

## RandomForest 예측

In [ ]:
# 4월 달은 test set에 저장
# 나머지 달은 train set에 저장
test = group[(group['year'] == 2020) & (group['month'] == 4)]
train = group[(group['year'] != 2020) | (group['month'] != 4)]
y_train = train['AMT']
X_train = train.drop(['AMT'], axis = 1)
y_test = test['AMT']
X_test = test.drop(['AMT'], axis = 1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf = RandomForestRegressor(random_state = 0, n_estimators = 10)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)

print("r2_score:", r2_score(y_test, pred))
print("MSE:", mean_squared_error(y_test, pred))
print("RMSE:", mean_squared_error(y_test, pred) ** 0.5)

r2_score: 0.9647684675927021
MSE: 0.3313801501360943
RMSE: 0.575656277770072


## LGBM 예측

In [ ]:
# 4월 달은 test set에 저장
# 나머지 달은 train set에 저장
test = group[(group['year_2020'] == 1) & (group['month_04'] == 1)]
train = group[(group['year_2020'] != 1) | (group['month_04'] != 1)]
y_train = train['AMT']
X_train = train.drop(['AMT'], axis = 1)
y_test = test['AMT']
X_test = test.drop(['AMT'], axis = 1)
df_sub = df_sub.drop(['AMT'], axis = 1)

### BaysianPredict

In [ ]:
!pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization
import numpy as np

In [ ]:
# train_test_X_1
# Bayesian Optimization
def lgb_evaluate(
    colsample_bytree,
    learning_rate,
    max_depth,
    min_child_weight,
    num_leaves,
    scale_pos_weight,
    subsample,
):

    reg = LGBMRegressor(
        colsample_bytree=colsample_bytree,
        learning_rate=learning_rate,
        max_depth=int(round(max_depth)),
        min_child_weight=min_child_weight,
        num_leaves=int(round(num_leaves)),
        scale_pos_weight=scale_pos_weight,
        subsample=subsample,
        n_estimators=400,
        random_state=1,
    )

# scoring=RMSE
    # scores = cross_val_score(
    #     reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10
    # )
    # rmse_scores = -(np.sqrt(-scores))

    # return np.mean(rmse_scores)

# scoring=r2_score
    scores = cross_val_score(
        reg, X_train, y_train, scoring="r2", cv=10
    )

    return np.mean(scores)


def bayesOpt(X_train, y_train):
    global lgbBO_max

    lgbBO = BayesianOptimization(
        lgb_evaluate,
        {
            "colsample_bytree": (0.4, 1),
            "learning_rate": (0.01, 1),
            "max_depth": (2, 90),
            "min_child_weight": (0.01, 70),
            "num_leaves": (5, 90),
            "scale_pos_weight": (1, 10000),
            "subsample": (0.4, 1),
        },
        random_state=1,
    )
    lgbBO.maximize(init_points=5, n_iter=70, acq="ei", xi=0.01)
    lgbBO_max = lgbBO.max
    col = lgbBO_max['params']['colsample_bytree']
    lea = lgbBO_max["params"]["learning_rate"]
    max = int(round(lgbBO_max["params"]["max_depth"]))
    min = lgbBO_max["params"]["min_child_weight"]
    num = int(round(lgbBO_max["params"]["num_leaves"]))
    sca = lgbBO_max["params"]["scale_pos_weight"]
    sub = lgbBO_max["params"]["subsample"]
    print('colsample_bytree :',lgbBO_max['params']['colsample_bytree'])
    print('learning_rate :',lgbBO_max["params"]["learning_rate"])
    print('max_depth :',int(round(lgbBO_max["params"]["max_depth"])))
    print('min_child_weight :', lgbBO_max["params"]["min_child_weight"])
    print('num_leaves :', int(round(lgbBO_max["params"]["num_leaves"])))
    print('scale_pos_weight :', lgbBO_max["params"]["scale_pos_weight"])
    print('subsample :', lgbBO_max["params"]["subsample"])


# X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2)
bayesOpt(X_train, y_train)

In [ ]:
# LGBMRegressor
# reg = LGBMRegressor(
#     colsample_bytree=lgbBO_max["params"]["colsample_bytree"],
#     learning_rate=lgbBO_max["params"]["learning_rate"],
#     max_depth=int(round(lgbBO_max["params"]["max_depth"])),
#     min_child_weight=lgbBO_max["params"]["min_child_weight"],
#     num_leaves=int(round(lgbBO_max["params"]["num_leaves"])),
#     scale_pos_weight=lgbBO_max["params"]["scale_pos_weight"],
#     subsample=lgbBO_max["params"]["subsample"],
#     n_estimators=400,
#     random_state=1,
# ).fit(X_train, y_train)
reg = LGBMRegressor(
    colsample_bytree= 0.4425400901816957,
    learning_rate= 0.23058334175622,
    max_depth= 39,
    min_child_weight= 0.2832506213695897,
    num_leaves= 89,
    scale_pos_weight= 4367.869754461021,
    subsample= 0.48365308414211655,
    n_estimators=400,
    random_state=1,
).fit(X_train, y_train)
predictions = reg.predict(X_test)

print("r2_score:", r2_score(y_test, predictions))
print()
print("MSE:", mean_squared_error(y_test, predictions))
print("RMSE:", mean_squared_error(y_test, predictions) ** 0.5)

r2_score: 0.9744629868704011

MSE: 0.2401956050926976
RMSE: 0.490097546507527


In [ ]:
print(y_test.mean())

In [ ]:
# Bayesian Optimization
def lgb_evaluate(
    colsample_bytree,
    learning_rate,
    max_depth,
    min_child_weight,
    num_leaves,
    subsample,
    lambda_l1,
    lambda_l2,
):

    reg = LGBMRegressor(
        boosting_type="gbdt",
        colsample_bytree=colsample_bytree,
        learning_rate=learning_rate,
        max_depth=int(round(max_depth)),
        min_child_weight=min_child_weight,
        num_leaves=int(round(num_leaves)),
        subsample=subsample,
        lambda_l1=lambda_l1,
        lambda_l2=lambda_l2,
        n_estimators=3000,
        random_state=1,
    )

    # scoring=RMSE
    fit_params = {
        "early_stopping_rounds": 15,
        "eval_metric": "rmse",
        "verbose": False,
        "eval_set": [[X_test, y_test]],
    }

    scores = cross_val_score(
        reg,
        X_train,
        y_train,
        scoring="neg_mean_squared_error",
        cv=10,
        fit_params=fit_params,
    )
    rmse_scores = -(np.sqrt(-scores))

    return np.mean(rmse_scores)


#     # scoring=r2_score
#     fit_params = {
#         "early_stopping_rounds": 15,
#         "eval_metric": "auc",
#         "verbose": False,
#         "eval_set": [[X_test, y_test]],
#     }


#     scores = cross_val_score(
#         reg, X_train, y_train, scoring="r2", cv=10, fit_params=fit_params
#     )

#     return np.mean(scores)


def bayesOpt(X_train, y_train):
    global lgbBO_max

    lgbBO = BayesianOptimization(
        lgb_evaluate,
        {
            "colsample_bytree": (0.4, 1),
            "learning_rate": (0.01, 1),
            "max_depth": (2, 90),
            "min_child_weight": (0.01, 70),
            "num_leaves": (5, 90),
            "subsample": (0.4, 1),
            "lambda_l1": (0, 1),
            "lambda_l2": (0, 1),
        },
        random_state=1,
    )
    lgbBO.maximize(init_points=5, n_iter=100, acq="ei", xi=0.01)
    lgbBO_max = lgbBO.max


# X_train, X_test, y_train, y_test = train_test_split(
#     train_X, train_y, test_size=0.2, shuffle=True, random_state=1
# )
bayesOpt(X_train, y_train)

In [ ]:
# LGBMRegressor <- 최적의 n_estimators 찾기
lgbBO_max["params"]["max_depth"] = int(round(lgbBO_max["params"]["max_depth"]))
lgbBO_max["params"]["num_leaves"] = int(round(lgbBO_max["params"]["num_leaves"]))
params = lgbBO_max["params"]

reg = LGBMRegressor(
    boosting_type="gbdt", **lgbBO_max["params"], n_estimators=5000, random_state=1,
).fit(
    X_train,
    y_train,
    early_stopping_rounds=15,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_metric="root_mean_squared_error",
    verbose=True,
)

predictions = reg.predict(X_test)
print("r2_score:", r2_score(y_test, predictions))
print("MSE:", mean_squared_error(y_test, predictions))
print("RMSE:", mean_squared_error(y_test, predictions) ** 0.5)

In [ ]:
# LGBMRegressor <- 전체 데이터 학습
hyper_params = reg.get_params()
hyper_params["n_estimators"] = reg.best_iteration_
reg = LGBMRegressor(**hyper_params).fit(train_X, train_y,)

predictions = reg.predict(X_test)
print("r2_score:", r2_score(y_test, predictions))
print()
print("MSE:", mean_squared_error(y_test, predictions))
print("RMSE:", mean_squared_error(y_test, predictions) ** 0.5)

## XGBoost

In [ ]:
# 4월 달은 test set에 저장
# 나머지 달은 train set에 저장
test = group[(group['year_2020'] == 1) & (group['month_04'] == 1)]
train = group[(group['year_2020'] != 1) | (group['month_04'] != 1)]
y_train = train['AMT']
X_train = train.drop(['AMT'], axis = 1)
y_test = test['AMT']
X_test = test.drop(['AMT'], axis = 1)
df_sub = df_sub.drop(['AMT'], axis = 1)

### BaysianPredict

In [ ]:
!pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization
import numpy as np

In [ ]:
# train_test_X_1
# Bayesian Optimization
def lgb_evaluate(
    colsample_bytree,
    learning_rate,
    max_depth,
    min_child_weight,
    num_leaves,
    scale_pos_weight,
    subsample,
):

    reg = XGBRegressor(
        colsample_bytree=colsample_bytree,
        learning_rate=learning_rate,
        max_depth=int(round(max_depth)),
        min_child_weight=min_child_weight,
        num_leaves=int(round(num_leaves)),
        scale_pos_weight=scale_pos_weight,
        subsample=subsample,
        n_estimators=100,
        random_state=1, 
    )

# scoring=RMSE
    # scores = cross_val_score(
    #     reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10
    # )
    # rmse_scores = -(np.sqrt(-scores))

    # return np.mean(rmse_scores)

# scoring=r2_score
    scores = cross_val_score(
        reg, X_train, y_train, scoring="r2", cv=10
    )

    return np.mean(scores)


def bayesOpt(X_train, y_train):
    global xgbBO_max

    xgbBO = BayesianOptimization(
        lgb_evaluate,
        {
            "colsample_bytree": (0.4, 1),
            "learning_rate": (0.01, 1),
            "max_depth": (2, 90),
            "min_child_weight": (0.01, 70),
            "num_leaves": (5, 90),
            "scale_pos_weight": (1, 10000),
            "subsample": (0.4, 1),
        },
        random_state=1,
    )
    xgbBO.maximize(init_points=5, n_iter=20, acq="ei", xi=0.01)
    xgbBO_max = xgbBO.max
    col = xgbBO_max['params']['colsample_bytree']
    lea = xgbBO_max["params"]["learning_rate"]
    max = int(round(xgbBO_max["params"]["max_depth"]))
    min = xgbBO_max["params"]["min_child_weight"]
    num = int(round(xgbBO_max["params"]["num_leaves"]))
    sca = xgbBO_max["params"]["scale_pos_weight"]
    sub = xgbBO_max["params"]["subsample"]
    print('colsample_bytree :',xgbBO_max['params']['colsample_bytree'])
    print('learning_rate :',xgbBO_max["params"]["learning_rate"])
    print('max_depth :',int(round(xgbBO_max["params"]["max_depth"])))
    print('min_child_weight :', xgbBO_max["params"]["min_child_weight"])
    print('num_leaves :', int(round(xgbBO_max["params"]["num_leaves"])))
    print('scale_pos_weight :', xgbBO_max["params"]["scale_pos_weight"])
    print('subsample :', xgbBO_max["params"]["subsample"])


# X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2)
bayesOpt(X_train, y_train)

In [ ]:
# XGBRegressor
reg = XGBRegressor(
    colsample_bytree=xgbBO_max["params"]["colsample_bytree"],
    learning_rate=xgbBO_max["params"]["learning_rate"],
    max_depth=int(round(xgbBO_max["params"]["max_depth"])),
    min_child_weight=xgbBO_max["params"]["min_child_weight"],
    num_leaves=int(round(xgbBO_max["params"]["num_leaves"])),
    scale_pos_weight=xgbBO_max["params"]["scale_pos_weight"],
    subsample=xgbBO_max["params"]["subsample"],
    n_estimators=400,
    random_state=1,
).fit(X_train, y_train)
# reg = XGBRegressor(
#     colsample_bytree=col,
#     learning_rate=lea,
#     max_depth=max,
#     min_child_weight=min,
#     num_leaves=num,
#     scale_pos_weight=sca,
#     subsample=sub,
#     n_estimators=400,
#     random_state=1,
# ).fit(X_train, y_train)
predictions = reg.predict(X_test)

print("r2_score:", r2_score(y_test, predictions))
print()
print("MSE:", mean_squared_error(y_test, predictions))
print("RMSE:", mean_squared_error(y_test, predictions) ** 0.5)

### 제출용

In [ ]:
df_sub['month_01'] = 0
df_sub['month_02'] = 0
df_sub['month_03'] = 0
df_sub['month_05'] = 0
df_sub['month_06'] = 0
df_sub['month_08'] = 0
df_sub['month_09'] = 0
df_sub['month_10'] = 0
df_sub['month_11'] = 0
df_sub['month_12'] = 0

In [ ]:
df_sub['year_2019'] = 0

In [ ]:
pred = reg.predict(df_sub)

In [ ]:
pred = np.expm1(pred)
pred = np.round(pred, 0)

In [ ]:
len(pred)

In [ ]:
result = pd.read_csv(path + 'submission.csv'); result

In [ ]:
result['AMT'] = pred

In [ ]:
result.to_csv('./result2.csv', encoding = 'utf-8-sig')